In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('consolidados/contaminantes_2010_2024.csv',sep=',', encoding='utf-8')
df.head()

,FECHA,ANIO,HORA,MES,DIA,PM10,PM2.5,CO,NO2,O3,SO2,LOCALIDAD,ID_LOCALIDAD
0,20201001,2020,0,10,1,26.8,10.3,114.49,14.29104,38.6514,13.87858,Usme,5
1,20201001,2020,1,10,1,23.6,7.7,228.98,14.10300,42.5754,22.51996,Usme,5
2,20201001,2020,2,10,1,24.5,NaN,114.49,7.70964,46.8918,8.37952,Usme,5
3,20201001,2020,3,10,1,18.0,NaN,114.49,14.85516,39.6324,10.21254,Usme,5
4,20201001,2020,4,10,1,13.6,NaN,228.98,29.89836,25.8984,38.23156,Usme,5


In [3]:
#Localidades son de 1 a 20, revisar que localidades no tienen datos
localidades = df["ID_LOCALIDAD"].unique()
localidades.sort()

localidades_sin_datos = []
for i in range(1,21):
    if i not in localidades:
        localidades_sin_datos.append(i)
        
print("Localidades sin datos:", localidades_sin_datos)

Localidades sin datos: [3, 14, 17, 18, 20]


In [4]:
#Solo vamos a agregar datos con triangulación para el periodo 2020-2024, pues de 2010-2019 solo hay datos de PM10 de 9 localidades. 
fechas=df['FECHA'].unique()
fechas.sort()
fechas=fechas[(fechas >= 20200101)]
fechas

array([20200101, 20200102, 20200103, ..., 20241229, 20241230, 20241231],
      shape=(1827,))

In [5]:
df[(df['ID_LOCALIDAD'].isin([3,14,17,18,20]))].head(10)

,FECHA,ANIO,HORA,MES,DIA,PM10,PM2.5,CO,NO2,O3,SO2,LOCALIDAD,ID_LOCALIDAD


In [6]:
#Localidad 3, 14, 17 - Santa Fe, Los Mártires, La Candelaria
# Se deben agregar datos haciendo una media con las 3 estaciones más cercanas para formar triangulo 2-16-4.

#Localidad 18 - Rafael Uribe
# Se deben agregar datos haciendo una media con las 4 estaciones más cercanas para formar cuadrado 4-5-6-15.

#Localidad 20 - Sumapaz
# Se deben agregar datos haciendo una media con las dos estaciones más cercanas, (no hay manera de formar un poligóno) 5-19.
'''
contaminantes=['PM10', 'PM2.5', 'CO', 'NO2', 'O3', 'SO2']

for fecha in fechas:
    for hora in range(0,24):
        datos_vecinos = df[(df['ID_LOCALIDAD'].isin([2,16,4])) & (df['FECHA'] == fecha) & (df['HORA'] == hora)]
        datos_vecinos_2 = df[(df['ID_LOCALIDAD'].isin([4,5,6,15])) & (df['FECHA'] == fecha) & (df['HORA'] == hora)]
        datos_vecinos_3 = df[(df['ID_LOCALIDAD'].isin([5,19])) & (df['FECHA'] == fecha) & (df['HORA'] == hora)]
    
        if len(datos_vecinos) > 0:
            nuevo_registro_santa_fe = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos['ANIO'].iloc[0],
                'MES': datos_vecinos['MES'].iloc[0],
                'DIA': datos_vecinos['DIA'].iloc[0],
                'LOCALIDAD': 'Santa Fe',
                'ID_LOCALIDAD': 3
            }
            nuevo_registro_martires = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos['ANIO'].iloc[0],
                'MES': datos_vecinos['MES'].iloc[0],
                'DIA': datos_vecinos['DIA'].iloc[0],
                'LOCALIDAD': 'Los Mártires',
                'ID_LOCALIDAD': 14
            }
            nuevo_registro_la_candelaria = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos['ANIO'].iloc[0],
                'MES': datos_vecinos['MES'].iloc[0],
                'DIA': datos_vecinos['DIA'].iloc[0],
                'LOCALIDAD': 'La Candelaria',
                'ID_LOCALIDAD': 17
            }

            for contaminante in contaminantes:
                suma=0.0
                cantidad_valores=0
                for dato in datos_vecinos[contaminante]:
                    if dato>0:
                        suma+=dato
                        cantidad_valores+=1
                        print(cantidad_valores)
                        print(dato)
                
                if cantidad_valores>0:
                    print("entro")
                    nuevo_registro_santa_fe[contaminante] = round(suma/cantidad_valores,3)
                    print(round(suma/cantidad_valores,3))
                    nuevo_registro_martires[contaminante] = round(suma/cantidad_valores,3)
                    nuevo_registro_la_candelaria[contaminante] = round(suma/cantidad_valores,3)


                else:
                    nuevo_registro_santa_fe[contaminante] = np.nan
                    nuevo_registro_martires[contaminante] = np.nan
                    nuevo_registro_la_candelaria[contaminante] = np.nan

            df = pd.concat([df, pd.DataFrame([nuevo_registro_santa_fe, nuevo_registro_martires, nuevo_registro_la_candelaria])], ignore_index=True)


        if len(datos_vecinos_2) > 0:
            nuevo_registro_rafael_uribe = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos_2['ANIO'].iloc[0],
                'MES': datos_vecinos_2['MES'].iloc[0],
                'DIA': datos_vecinos_2['DIA'].iloc[0],
                'LOCALIDAD': 'Rafael Uribe',
                'ID_LOCALIDAD': 18
            }
            for contaminante in contaminantes:
                promedio=0.0
                cantidad_valores=0
                for dato in datos_vecinos_2[contaminante]:
                    if dato!=np.nan:
                        promedio+=dato
                        cantidad_valores+=1
                if cantidad_valores>0:
                    nuevo_registro_rafael_uribe[contaminante] = round(suma/cantidad_valores,3)
                    
                else:
                    nuevo_registro_rafael_uribe[contaminante] = np.nan
                    
            df = pd.concat([df, pd.DataFrame([nuevo_registro_rafael_uribe])], ignore_index=True)

        if len(datos_vecinos_3) > 0:
            nuevo_registro_sumapaz = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos_3['ANIO'].iloc[0],
                'MES': datos_vecinos_3['MES'].iloc[0],
                'DIA': datos_vecinos_3['DIA'].iloc[0],
                'LOCALIDAD': 'Sumapaz',
                'ID_LOCALIDAD': 20
            }
            for contaminante in contaminantes:
                promedio=0.0
                cantidad_valores=0
                for dato in datos_vecinos_3[contaminante]:
                    if dato!=np.nan:
                        promedio+=dato
                        cantidad_valores+=1
                if cantidad_valores>0:
                    nuevo_registro_sumapaz[contaminante] = round(suma/cantidad_valores,3)
                    
                else:
                    nuevo_registro_sumapaz[contaminante] = np.nan
                    
            df = pd.concat([df, pd.DataFrame([nuevo_registro_sumapaz])], ignore_index=True)
'''

'\ncontaminantes=[\'PM10\', \'PM2.5\', \'CO\', \'NO2\', \'O3\', \'SO2\']\n\nfor fecha in fechas:\n    for hora in range(0,24):\n        datos_vecinos = df[(df[\'ID_LOCALIDAD\'].isin([2,16,4])) & (df[\'FECHA\'] == fecha) & (df[\'HORA\'] == hora)]\n        datos_vecinos_2 = df[(df[\'ID_LOCALIDAD\'].isin([4,5,6,15])) & (df[\'FECHA\'] == fecha) & (df[\'HORA\'] == hora)]\n        datos_vecinos_3 = df[(df[\'ID_LOCALIDAD\'].isin([5,19])) & (df[\'FECHA\'] == fecha) & (df[\'HORA\'] == hora)]\n\n        if len(datos_vecinos) > 0:\n            nuevo_registro_santa_fe = {\n                \'FECHA\': fecha,\n                \'HORA\': hora,\n                \'ANIO\': datos_vecinos[\'ANIO\'].iloc[0],\n                \'MES\': datos_vecinos[\'MES\'].iloc[0],\n                \'DIA\': datos_vecinos[\'DIA\'].iloc[0],\n                \'LOCALIDAD\': \'Santa Fe\',\n                \'ID_LOCALIDAD\': 3\n            }\n            nuevo_registro_martires = {\n                \'FECHA\': fecha,\n              

In [7]:
#Localidad 3, 14, 17 - Santa Fe, Los Mártires, La Candelaria
# Se deben agregar datos haciendo una media con las 3 estaciones más cercanas para formar triangulo 2-16-4.

#Localidad 18 - Rafael Uribe
# Se deben agregar datos haciendo una media con las 4 estaciones más cercanas para formar cuadrado 4-5-6-15.

#Localidad 20 - Sumapaz
# Se deben agregar datos haciendo una media con las dos estaciones más cercanas, (no hay manera de formar un poligóno) 5-19.

for fecha in fechas:
    print(fecha)
    for hora in range(0,24):
        datos_vecinos = df[(df['ID_LOCALIDAD'].isin([2,16,4])) & (df['FECHA'] == fecha) & (df['HORA'] == hora)]
        datos_vecinos_2 = df[(df['ID_LOCALIDAD'].isin([4,5,6,15])) & (df['FECHA'] == fecha) & (df['HORA'] == hora)]
        datos_vecinos_3 = df[(df['ID_LOCALIDAD'].isin([5,19])) & (df['FECHA'] == fecha) & (df['HORA'] == hora)]

        if len(datos_vecinos) > 0:
            nuevo_registro_santa_fe = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos['ANIO'].iloc[0],
                'MES': datos_vecinos['MES'].iloc[0],
                'DIA': datos_vecinos['DIA'].iloc[0],
                'PM10': datos_vecinos['PM10'].mean(),
                'PM2.5': datos_vecinos['PM2.5'].mean(),
                'NO2': datos_vecinos['NO2'].mean(),
                'O3': datos_vecinos['O3'].mean(),
                'CO': datos_vecinos['CO'].mean(),
                'SO2': datos_vecinos['SO2'].mean(),
                'LOCALIDAD': 'Santa Fe',
                'ID_LOCALIDAD': 3
            }
            nuevo_registro_martires = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos['ANIO'].iloc[0],
                'MES': datos_vecinos['MES'].iloc[0],
                'DIA': datos_vecinos['DIA'].iloc[0],
                'PM10': datos_vecinos['PM10'].mean(),
                'PM2.5': datos_vecinos['PM2.5'].mean(),
                'NO2': datos_vecinos['NO2'].mean(),
                'O3': datos_vecinos['O3'].mean(),
                'CO': datos_vecinos['CO'].mean(),
                'SO2': datos_vecinos['SO2'].mean(),
                'LOCALIDAD': 'Los Mártires',
                'ID_LOCALIDAD': 14
            }
            nuevo_registro_la_candelaria = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos['ANIO'].iloc[0],
                'MES': datos_vecinos['MES'].iloc[0],
                'DIA': datos_vecinos['DIA'].iloc[0],
                'PM10': datos_vecinos['PM10'].mean(),
                'PM2.5': datos_vecinos['PM2.5'].mean(),
                'NO2': datos_vecinos['NO2'].mean(),
                'O3': datos_vecinos['O3'].mean(),
                'CO': datos_vecinos['CO'].mean(),
                'SO2': datos_vecinos['SO2'].mean(),
                'LOCALIDAD': 'La Candelaria',
                'ID_LOCALIDAD': 17
            }
            df = pd.concat([df, pd.DataFrame([nuevo_registro_santa_fe, nuevo_registro_martires, nuevo_registro_la_candelaria])], ignore_index=True)

        if len(datos_vecinos_2) > 0:
            nuevo_registro_rafael_uribe = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos_2['ANIO'].iloc[0],
                'MES': datos_vecinos_2['MES'].iloc[0],
                'DIA': datos_vecinos_2['DIA'].iloc[0],
                'PM10': datos_vecinos_2['PM10'].mean(),
                'PM2.5': datos_vecinos_2['PM2.5'].mean(),
                'NO2': datos_vecinos_2['NO2'].mean(),
                'O3': datos_vecinos_2['O3'].mean(),
                'CO': datos_vecinos_2['CO'].mean(),
                'SO2': datos_vecinos_2['SO2'].mean(),
                'LOCALIDAD': 'Rafael Uribe',
                'ID_LOCALIDAD': 18
            }
            df = pd.concat([df, pd.DataFrame([nuevo_registro_rafael_uribe])], ignore_index=True)

        if len(datos_vecinos_3) > 0:
            nuevo_registro_sumapaz = {
                'FECHA': fecha,
                'HORA': hora,
                'ANIO': datos_vecinos_3['ANIO'].iloc[0],
                'MES': datos_vecinos_3['MES'].iloc[0],
                'DIA': datos_vecinos_3['DIA'].iloc[0],
                'PM10': datos_vecinos_3['PM10'].mean(),
                'PM2.5': datos_vecinos_3['PM2.5'].mean(),
                'NO2': datos_vecinos_3['NO2'].mean(),
                'O3': datos_vecinos_3['O3'].mean(),
                'CO': datos_vecinos_3['CO'].mean(),
                'SO2': datos_vecinos_3['SO2'].mean(),
                'LOCALIDAD': 'Sumapaz',
                'ID_LOCALIDAD': 20
            }
            df = pd.concat([df, pd.DataFrame([nuevo_registro_sumapaz])], ignore_index=True)


20200101
20200102
20200103
20200104
20200105
20200106
20200107
20200108
20200109
20200110
20200111
20200112
20200113
20200114
20200115
20200116
20200117
20200118
20200119
20200120
20200121
20200122
20200123
20200124
20200125
20200126
20200127
20200128
20200129
20200130
20200131
20200201
20200202
20200203
20200204
20200205
20200206
20200207
20200208
20200209
20200210
20200211
20200212
20200213
20200214
20200215
20200216
20200217
20200218
20200219
20200220
20200221
20200222
20200223
20200224
20200225
20200226
20200227
20200228
20200229
20200301
20200302
20200303
20200304
20200305
20200306
20200307
20200308
20200309
20200310
20200311
20200312
20200313
20200314
20200315
20200316
20200317
20200318
20200319
20200320
20200321
20200322
20200323
20200324
20200325
20200326
20200327
20200328
20200329
20200330
20200331
20200401
20200402
20200403
20200404
20200405
20200406
20200407
20200408
20200409
20200410
20200411
20200412
20200413
20200414
20200415
20200416
20200417
20200418
20200419
20200420
2

In [8]:
#Chequeo de que calculo correctamente los valores

df[(df['ID_LOCALIDAD'].isin([2,16,4,3,17,14])) & (df['FECHA'] == 20201001) & (df['HORA'] == 0)]


,FECHA,ANIO,HORA,MES,DIA,PM10,PM2.5,CO,NO2,O3,SO2,LOCALIDAD,ID_LOCALIDAD
1023631,20201001,2020,0,10,1,23.2,14.000000,1030.41,NaN,NaN,NaN,Chapinero,2
1173914,20201001,2020,0,10,1,26.9,19.000000,801.43,49.45452,2.9430,1.3093,Puente Aranda,16
1341581,20201001,2020,0,10,1,6.0,2.800000,NaN,NaN,26.0946,NaN,San Cristóbal,4
1768711,20201001,2020,0,10,1,18.7,11.933333,915.92,49.45452,14.5188,1.3093,Santa Fe,3
1768712,20201001,2020,0,10,1,18.7,11.933333,915.92,49.45452,14.5188,1.3093,Los Mártires,14
1768713,20201001,2020,0,10,1,18.7,11.933333,915.92,49.45452,14.5188,1.3093,La Candelaria,17


In [ ]:
#Exportamos el df en un csv
df.to_csv(
    "consolidados/contaminantes_2010_2024_con_localidades_faltantes.csv",
    encoding="utf-8-sig",   # incluye BOM compatible con Power BI
    index=False,            # sin índice extra
    sep=",",                # separador por comas
    decimal="."             # punto decimal
)
len(fechas)

1827